In [1]:
"""
JobScheduler: a basic version of the job-shop problem, solved using the
OrTools library.
"""

import random

from ortools.constraint_solver import pywrapcp
import numpy

NUM_OF_RESOURCES = 20
TIME_FRAME = 20
MIN_JOB_LENGTH = 3
MAX_JOB_LENGTH = 10
NUM_COMPATIBLE_RESOURCES_PER_JOB = 2
SCHEDULING_HORIZON = 3 * TIME_FRAME
MAX_NUM_OF_SOLUTIONS = 20
NUM_OF_JOBS = 100

################################################################################
# Classes, functions and initial setup
class Job(object):
    """A job to be scheduled"""
    def __init__(self, code, arrival_date, compatible_resources, length):
        self.code = code
        self.arrival_date = arrival_date
        self.compatible_resources = compatible_resources
        self.length = length

def create_random_jobs(num_of_jobs):
    jobs = list()
    for i in range(num_of_jobs):
        code = i
        arrival_date = random.choice(range(TIME_FRAME))
        compatible_resources = random.sample(
            resources, NUM_COMPATIBLE_RESOURCES_PER_JOB
        )
        length = random.randint(MIN_JOB_LENGTH, MAX_JOB_LENGTH)
        jobs.append(Job(code, arrival_date, compatible_resources, length))
    return jobs

def display_schedule(schedule):
    for resource in xrange(schedule.shape[0]):
        row = '{}: '.format(str(int(resource)).zfill(2))
        for date in xrange(schedule.shape[1]):
            code = schedule[resource, date]
            if numpy.isnan(code):
                row += u'□'
            else:
                colour = '\x1b[{};3{}m'.format(int(code) % 2, int(code) % 8)
                row += colour + u'■' + '\x1b[0m'
        print row

resources = range(NUM_OF_RESOURCES)
jobs = create_random_jobs(NUM_OF_JOBS)
solver = pywrapcp.Solver('JobScheduler')

################################################################################
# Variables: use `FixedDurationIntervalVar` variables for shifts
shift_grid = dict()
for job in jobs:
    for resource in job.compatible_resources:
        shift_grid[(job, resource)] = solver.FixedDurationIntervalVar(
            job.arrival_date,
            SCHEDULING_HORIZON,
            job.length,
            True,
            'shift for job {} / resource {}'.format(job.code, resource)
        )

################################################################################
# Constraints: each job is performed exactly once
for job in jobs:
    shifts_for_current_job = [
        shift_grid[key] for key in shift_grid.keys() if key[0] == job
    ]
    solver.Add(
        solver.Sum(
            [shift.PerformedExpr() for shift in shifts_for_current_job]
        ) == 1
    )

################################################################################
# Constraints: each resource can only be used by one job at the time
shifts_grouped_by_resource = dict()
for job_resource_pair, shift in shift_grid.items():
    resource = job_resource_pair[1]
    try:
        shifts_grouped_by_resource[resource].add(shift)
    except KeyError:
        shifts_grouped_by_resource[resource] = set()
        shifts_grouped_by_resource[resource].add(shift)
for resource, shift_set in shifts_grouped_by_resource.items():
    disj = solver.DisjunctiveConstraint(
        shift_set, 'shifts for resource {}'.format(resource)
    )
    solver.Add(disj)

################################################################################
# Solving stage
solution = solver.Assignment()
shift_list = shift_grid.values()
solution.Add(shift_list)
db = solver.Phase(shift_list, solver.INTERVAL_DEFAULT)
solver.NewSearch(db)

num_solutions = 0
while solver.NextSolution() and num_solutions < MAX_NUM_OF_SOLUTIONS:
    num_solutions += 1
    schedule = numpy.empty(shape=(NUM_OF_RESOURCES, SCHEDULING_HORIZON))
    # Initialise to `NaN`
    schedule[:] = numpy.nan
    for job in jobs:
        for resource in job.compatible_resources:
            shift_var = shift_grid[(job, resource)]
            if shift_var.MustBePerformed():
                start_date = int(shift_var.StartMin())
                end_date = start_date + job.length
                for date in range(start_date, end_date):
                    schedule[resource, date] = job.code
    display_schedule(schedule)
    print 80 * '-'

00: □■■■■■■■■■■■■■■■■■■■■■■■■■■■□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□
01: □■■■■■■□□□■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■□□□□□□□□□□□□□□□□□□
02: □■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■□□□□□□□□□□□□□□□□□□□□□□□
03: □□□■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■□□□□□□□□□□□□□□□□□□□
04: □■■■■■□□□□□■■■■■■■■■■■■■■■■■■■■■■■■■■□□□□□□□□□□□□□□□□□□□□□□□
05: ■■■□■■■■■■■■■■■■■■■■■■■■■■■■■■□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□
06: □■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■□□□□□□□□□□□□□□□□□□□□□
07: □■■■■■■■□□□□□□■■■■■■■■■■■■■■■■■■■□□□□□□□□□□□□□□□□□□□□□□□□□□□
08: □□■■■■■■■■■■■■■■■■■■■■■■■■■■■□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□
09: □□□□■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■□□□□□□□□□□□□□□□□□□□
10: □□■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■□□□□□□□□□
11: ■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■□□□□□□□□□□□□□□□□□□□□□□
12: □■■■■■■■■■■■■■□□■■■■■■■■■■■■■■■■■□□□□□□□□□□□□□□□□□□□□□□□□□□□
13: □□■■■■■■■■■■■■■■■■■■■■□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□
14: ■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■□□□□□□□□□□□□□□□□□□□□
15: □□■■■■■■■■■■■■■■■■■■■